In [22]:
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade pandas


Requirement already up-to-date: google-cloud-bigquery in /opt/conda/lib/python3.7/site-packages (1.24.0)
Requirement already up-to-date: pandas in /opt/conda/lib/python3.7/site-packages (1.0.3)


## We have already injested the preprcessed data into Big query and created a dataset called as covid19

In [2]:
from google.cloud import bigquery
import pandas as pd
client = bigquery.Client()

In [3]:

dataset = bigquery.Dataset(client.dataset('covid19'))
dataset.location = 'US'

In [4]:
client.get_dataset("covid19")


Dataset(DatasetReference('covid-19-kbs-project', 'covid19'))

In [5]:
%load_ext google.cloud.bigquery


The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## Fetch only confirmed cases from the dataset

### Store the result in a new table covid19.covid19_confirmed

In [8]:
%%bigquery
CREATE TABLE `covid-19-kbs-project.covid19.covid19_confirmed`
  AS
SELECT * FROM `covid-19-kbs-project.covid19.covid19_data_table` where Case_Type = "Confirmed";

""


In [9]:
%%bigquery
SELECT count(*) FROM `covid-19-kbs-project.covid19.covid19_confirmed`;

,f0_
0,359040


### There are 359040 confirmed cases of Covid 19 untill 5th May 2020.

# Creating Training and Testing data from the dataset

#### Creating the training dataset.

In [12]:
# Creating training data of 250000 records. (train:test --> 70:30 split ratio)
train_sql = "SELECT * FROM `covid-19-kbs-project.covid19.covid19_confirmed` \
ORDER BY Date \
LIMIT 250000"

In [18]:
df = client.query(train_sql).result().to_dataframe()

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Case_Type           250000 non-null  object 
 1   Cases               250000 non-null  int64  
 2   Difference          250000 non-null  int64  
 3   Date                250000 non-null  object 
 4   Country_Region      250000 non-null  object 
 5   Province_State      237073 non-null  object 
 6   County_name_USonly  231108 non-null  object 
 7   Lat                 242544 non-null  float64
 8   Long                242544 non-null  float64
 9   Continent           250000 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 19.1+ MB


#### IMP NOTE: Remember to remove the headers and always keep the target variable at the statring position.

In [24]:
df.Date = pd.to_datetime(df.Date)
df.Cases = pd.to_numeric(df.Cases)
df = df [['Cases','Case_Type',  'Difference', 'Date', 'Country_Region',
       'Province_State', 'County_name_USonly', 'Lat', 'Long', 'Continent']]
df.to_csv('covid19_confirmed_train.csv',header=False,index = False)

#### Creating the testing dataset.

In [25]:
# Creating testing data of 70000 records. (train:test --> 70:30 split ratio)
test_sql = "SELECT * FROM `covid-19-kbs-project.covid19.covid19_confirmed` ORDER BY Date Desc LIMIT 70000"

In [26]:
df_test = client.query(test_sql).result().to_dataframe()

In [27]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Case_Type           70000 non-null  object 
 1   Cases               70000 non-null  int64  
 2   Difference          70000 non-null  int64  
 3   Date                70000 non-null  object 
 4   Country_Region      70000 non-null  object 
 5   Province_State      66385 non-null  object 
 6   County_name_USonly  64715 non-null  object 
 7   Lat                 67906 non-null  float64
 8   Long                67906 non-null  float64
 9   Continent           70000 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 5.3+ MB


In [28]:
df_test.Date =pd.to_datetime(df_test.Date)
df_test.Cases = pd.to_numeric(df_test.Cases)
df_test = df_test [['Case_Type',  'Difference', 'Date', 'Country_Region',
       'Province_State', 'County_name_USonly', 'Lat', 'Long', 'Continent']]
df_test.to_csv('covid19_confirmed_test.csv',header=False,index = False)